# PyTorch Tutorial - Sequence Models

In view of the AI quiz, capstone and other deliverables during week 5, the exercises for this lab is made relatively straightforward. The homework exercises can be found at the end of this tutorial.

## Sentiment detection with BERT

The task tackled here is sentiment detection of IMDB articles. In this notebook, we will utilize the Huggingface suite of libraries (transformers, datasets, evaluate, etc) to build upon it to perform some quick prototyping and evaluation of models. Some of the concepts we will use:
- loading pre-trained models
- fine-tuning on downstream tasks
- streamlining the training loop
- evaluation

In [ ]:
import torch
# Check torch version
print(torch.__version__)
# Check for CUDA
print(torch.cuda.is_available())
print(torch.version.cuda)

: 

First, we install and import various libraries that are useful for this task, namely:
- datasets: Contains numerous popular datasets of different modalities and sizes, along with useful pre-processing tools (https://github.com/huggingface/datasets)
- transformers: Contains numerous pre-trained model from the huggingface community, along with many useful pipelines for automating the training and evaluation process (https://github.com/huggingface/transformers)
- evaluate: Contains numerous standard and custom evaluation metrics that can be easily used as part of the evaluation pipeline (https://github.com/huggingface/evaluate)

In [2]:
!pip install -U evaluate
!pip install -U datasets
!pip install -U accelerate
!pip install -U transformers

import numpy as np
import pandas as pd
import evaluate
import accelerate
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer



  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
supervisely 6.73.212 requires pydantic<=2.8.2,>=1.7.4, but you have pydantic 2.10.6 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 67.9 MB/s eta 0:00:00


The evaluate library provides many standard evaluation metrics (such as f1, precision, accuracy, etc) for specific tasks. For example, the list of standard metrics for the text classification task can be found at https://huggingface.co/tasks/text-classification. Similarly, there are also many custom-made metrics for specific tasks, which the foillowing code prints out.

In [3]:
metrics_list = evaluate.list_evaluation_modules()
print(metrics_list)

['codeparrot/apps_metric', 'lvwerra/test', 'angelina-wang/directional_bias_amplification', 'cpllab/syntaxgym', 'lvwerra/bary_score', 'hack/test_metric', 'yzha/ctc_eval', 'mfumanelli/geometric_mean', 'daiyizheng/valid', 'erntkn/dice_coefficient', 'mgfrantz/roc_auc_macro', 'Vlasta/pr_auc', 'gorkaartola/metric_for_tp_fp_samples', 'idsedykh/metric', 'idsedykh/codebleu2', 'idsedykh/codebleu', 'idsedykh/megaglue', 'Vertaix/vendiscore', 'GMFTBY/dailydialogevaluate', 'GMFTBY/dailydialog_evaluate', 'jzm-mailchimp/joshs_second_test_metric', 'ola13/precision_at_k', 'yulong-me/yl_metric', 'abidlabs/mean_iou', 'abidlabs/mean_iou2', 'KevinSpaghetti/accuracyk', 'NimaBoscarino/weat', 'ronaldahmed/nwentfaithfulness', 'Viona/infolm', 'kyokote/my_metric2', 'kashif/mape', 'Ochiroo/rouge_mn', 'leslyarun/fbeta_score', 'anz2/iliauniiccocrevaluation', 'zbeloki/m2', 'xu1998hz/sescore', 'dvitel/codebleu', 'NCSOFT/harim_plus', 'JP-SystemsX/nDCG', 'sportlosos/sescore', 'Drunper/metrica_tesi', 'jpxkqx/peak_signal_

We now load in the dataset of interest for this task, which is the IMDB dataset that contains a series of movie reviews and the corresponding sentiment label.

In [43]:
dataset = load_dataset("imdb")

In [44]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


##Transfer Learning

Transfer learning is technique where we take a model that has learnt to solve one problem, then apply it to solving a different but related problem. In this tutorial, we are applying transfer learning by taking the pre-trained BERT-tiny model and fine-tuning it on this task of sentiment detection on IMDB. Details on the BERT-tiny model can be found at https://huggingface.co/google/bert_uncased_L-2_H-128_A-2.

The following code loads a model checkpoint from huggingface (i.e., BERT-tiny as defined in the model_checkpoint variable), which you can use to further fine-tune on the specific downstream task (i.e., IMDB sentiment detection). Apart from a standard huggingface model checkpoint, you can also use this to load one of your previously trained model, e.g., to continue more training or transfer learn on another task. This code also performs some pre-processing of the dataset in terms of tokenization and padding/truncating the text to a specific sequence that BERT requires.

In [7]:
model_checkpoint = "google/bert_uncased_L-2_H-128_A-2"
max_len = 512

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(input_data):
    texts = input_data
    return tokenizer(texts["text"], max_length=max_len, padding="max_length", truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
encoded_dataset['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

AutoModelForSequenceClassification provides a high-level interface/wrapper around specific pre-trained models that you can use for general text classification tasks, including finetuning it based on various configurations. More details can be found at https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification.

In [9]:
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1)
model.resize_token_embeddings(len(tokenizer)) # need to resize due to new tokens added

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 128, padding_idx=0)

Here, TrainingArguments allow us to specific various parameters relating to the training of our model, including training epochs, batch sizes, etc. More details can be found at https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments.

In [11]:
metric_name = 'f1'
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"./snapshots/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    fp16=True
)

For computation of our evaluation metric

In [12]:
metric = evaluate.load(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="micro")

Now, we initialize a Trainer that handles the training loop based on our definition of the model and training parameters as defined in the earlier part of this tutorial.

In [13]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['train'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/var/folders/p0/6sw494x1303f8w2k35bxtgk80000gn/T/ipykernel_76527/3731775950.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'mps').

Start the actual model training.

In [15]:
train_log = trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
# trainer.save_model("./models/myFinetunedModel") # for saving your model

Finally, we perform our evaluation on our test set using the fine-tuned model from earlier.

In [16]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device="cuda:0")
results = classifier(dataset['test']['text'], max_length=max_len, padding="max_length", truncation=True)
dfResults = pd.DataFrame.from_dict(results)
dfResults['label'] = dfResults['label'].str.replace('LABEL_','')
f1 = metric.compute(predictions=dfResults['label'].tolist(), references=dataset['test']['label'], average='micro')
print(f1)

Device set to use cuda:0


AssertionError: Torch not compiled with CUDA enabled

## Homework Exercises
**Due: 14th Mar, 11:59pm**
<br>
<br>
Submit your homework as either: (i) an ipynb file with your results inside; or (ii) a python file and separate pdf discussing your results. Based on what you have done so far in this tutorial, repeat another set of experiments with the following changes:

(a) There is a potential error in the design of the training and testing/evaluation step in this tutorial. Identify what it is and describe how you will solve this.

(b) Pick another dataset from the datasets library that is not a binary classification task (see https://huggingface.co/datasets). You can sample a subset from this dataset to reduce compute time (see https://huggingface.co/docs/datasets/en/process).

(c) Fine-tune the same model but with a dropout of 0.2, over 3 epochs and report on accuracy scores, in addition to F1 scores.



Part a:

The potential error in the design of the training and testing/ evaluation step is caused due to the same data samples being used for both training and evaluation:

train_dataset=encoded_dataset['train'],
eval_dataset=encoded_dataset['train'],

To solve the issue, the evaluation step must use different dataset/ different samples as shown in the code below:

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Since the tutorial does not contain a split dataset, I will proceed to split the dataset and implement the trainer method as shown below. I also need to change some parameters in the args method as I do not have an NVIDIA GPU.


In [30]:
#Part a
#New Trainer Method

from datasets import Dataset
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

args = TrainingArguments(
    f"./snapshots/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    fp16=False  # Changed to False
)

train_data = encoded_dataset['train']
train_df = train_data.to_pandas()
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)




/opt/homebrew/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/p0/6sw494x1303f8w2k35bxtgk80000gn/T/ipykernel_76527/2739572784.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [21]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.2.0
False


Part b

In [71]:
from datasets import load_dataset

# Load AG News dataset (multi-class classification)
dataset_new = load_dataset("ag_news")

# Sample a subset to reduce compute time (e.g., 10% of the dataset)
small_train_dataset = dataset_new["train"].shuffle(seed=42).select(range(5000))
small_test_dataset = dataset_new["test"].shuffle(seed=42).select(range(1000))


train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [73]:
print(dataset_new["train"][0])

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


In [74]:
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)  # Change "text" to "content" if needed


In [76]:
print(dataset_new["train"].column_names)  # This will show all available column names


['text', 'label']


In [77]:
column_name = "content"  # Change this based on the actual dataset structure
def tokenize_function(examples):
    return tokenizer(examples[column_name], padding="max_length", truncation=True)


In [83]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, AutoConfig
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Load dataset (Choose a multi-class dataset instead of binary classification)
dataset_new = load_dataset("ag_news")  # Change this if needed

# Check dataset structure
print(dataset_new["train"][0])  # Print first example to verify column names
column_name = "text"  # Adjust based on dataset

# Load tokenizer
model_checkpoint = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
max_len = 512

# Tokenization function
def preprocess_function(examples):
    return tokenizer(examples[column_name], max_length=max_len, padding="max_length", truncation=True)

# Apply tokenization
encoded_dataset = dataset_new.map(preprocess_function, batched=True)

# Get number of unique labels
num_labels = len(set(dataset_new["train"]["label"]))

# Load model with custom dropout
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, hidden_dropout_prob=0.2)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, config=config)

# Compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")  # Weighted for multi-class
    return {"accuracy": accuracy, "f1": f1}

# Training arguments
args = TrainingArguments(
    output_dir=f"./snapshots/{model_checkpoint}-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Fine-tuning for 3 epochs
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    fp16=True
)

# Split dataset
train_dataset = encoded_dataset["train"]
val_dataset = encoded_dataset["test"]  # Using test set for validation

# Initialize Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate on validation set
eval_results = trainer.evaluate()
print(f"Final validation accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Final validation F1 score: {eval_results['eval_f1']:.4f}")


{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: fp16 mixed precision requires a GPU (not 'mps').